# Assignment 04
#### Big Data Tools - Spring 2024
#### Matt Austen

### Define functions and load configuration

In [1]:
from neo4j import GraphDatabase
import yaml

def loadConfig():
    with open('config.yaml', 'r') as file:
        return yaml.safe_load(file)
    
config = loadConfig()

def getNeo4jConnection():
    return GraphDatabase.driver(
        config['neo4j']['uri'],
        auth=(config['neo4j']['user'], config['neo4j']['password']))

def clearDatabase(driver):
    with driver.session() as session:
        session.run('MATCH (n) DETACH DELETE n')

def quickCypher(neo4jDriver, query, verbose=False):
    with neo4jDriver.session() as session:
        result = session.run(query)
        if verbose==True:
            for record in result:
                print(record)

### Establish nodes & relationships

In [2]:
# Create Order node
query1 = '''
    LOAD CSV WITH HEADERS
    FROM 'https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv' AS row
    MERGE (o:Order {order_id: toInteger(row.`Order ID`)})
    SET
    o.region = row.Region,
    o.country = row.Country,
    o.item_type = row.`Item Type`,
    o.sales_channel = row.`Sales Channel`,
    o.order_priority = row.`Order Priority`,
    o.order_date = row.`Order Date`,
    o.ship_date = row.`Ship Date`,
    o.units_sold = toInteger(row.UnitsSold),
    o.unit_price = toFloat(row.UnitPrice),
    o.unit_cost = toFloat(row.UnitCost),
    o.total_revenue = toFloat(row.TotalRevenue),
    o.total_cost = toFloat(row.TotalCost),
    o.total_profit = toFloat(row.TotalProfit);
    '''

# Create Region node
query2 = '''
    LOAD CSV WITH HEADERS
    FROM 'https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv' AS row
    MERGE (r:Region {name: row.Region})
    SET
    r.country = row.Country;
    '''

# Create Country node
query3 = '''
    LOAD CSV WITH HEADERS
    FROM 'https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv' AS row
    MERGE (c:Country {name: row.Country})
    '''

# Create relationships
query4 = '''
    LOAD CSV WITH HEADERS
    FROM 'https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv' AS row
    MATCH (o:Order {order_id: toInteger(row.`Order ID`)})
    MATCH (r:Region {name: row.Region})
    MATCH (c:Country {name: row.Country})
    MERGE (o)-[:ORDERED_IN]->(c)
    MERGE (o)-[:ORDERED_IN]->(r)
    MERGE (c)-[:IS_IN]->(r);
    '''

# Establish database connection and reset it
driver = getNeo4jConnection()
clearDatabase(driver)

# Execute queries
quickCypher(driver, query1)
quickCypher(driver, query2)
quickCypher(driver, query3)
quickCypher(driver, query4)
quickCypher(driver, 'MATCH (x) RETURN count(x) as count', verbose=True)

<Record count=186>


### Query #1: Write query to find the Total Sales per Region, Country. The output should contain  Total Units Sold, Total Revenue

In [3]:
query1 = '''
    MATCH (n:Order)
    RETURN SUM(n.units_sold), SUM(n.total_revenue), n.region
    '''

quickCypher(driver, query1, verbose=True)

<Record SUM(n.units_sold)=92606 SUM(n.total_revenue)=24225437.419999998 n.region='Sub-Saharan Africa'>
<Record SUM(n.units_sold)=60376 SUM(n.total_revenue)=24765127.250000004 n.region='Middle East and North Africa'>
<Record SUM(n.units_sold)=42328 SUM(n.total_revenue)=10711258.129999999 n.region='Australia and Oceania'>
<Record SUM(n.units_sold)=121002 SUM(n.total_revenue)=34964749.830000006 n.region='Europe'>
<Record SUM(n.units_sold)=113129 SUM(n.total_revenue)=28840812.189999998 n.region='Asia'>
<Record SUM(n.units_sold)=53641 SUM(n.total_revenue)=17570835.419999998 n.region='Central America and the Caribbean'>
<Record SUM(n.units_sold)=11728 SUM(n.total_revenue)=3611757.5199999996 n.region='North America'>


### Query #2: Average Profit by Item Type

In [4]:
query2 = '''
    MATCH (n:Order)
    RETURN AVG(n.total_profit), n.item_type
    '''

quickCypher(driver, query2, verbose=True)

<Record AVG(n.total_profit)=12220.553846153847 n.item_type='Fruits'>
<Record AVG(n.total_profit)=294846.912 n.item_type='Clothes'>
<Record AVG(n.total_profit)=360624.55 n.item_type='Meat'>
<Record AVG(n.total_profit)=64356.90545454546 n.item_type='Beverages'>
<Record AVG(n.total_profit)=675658.4375000001 n.item_type='Office Supplies'>
<Record AVG(n.total_profit)=952039.6741666667 n.item_type='Cosmetics'>
<Record AVG(n.total_profit)=264249.26 n.item_type='Snacks'>
<Record AVG(n.total_profit)=122308.46250000001 n.item_type='Personal Care'>
<Record AVG(n.total_profit)=955267.7200000001 n.item_type='Household'>
<Record AVG(n.total_profit)=232570.91999999998 n.item_type='Vegetables'>
<Record AVG(n.total_profit)=325476.6533333333 n.item_type='Baby Food'>
<Record AVG(n.total_profit)=506911.9800000001 n.item_type='Cereal'>


### Query #3: Total profit by Sales Channel (Offline,Online)

In [5]:
query3 = '''
    MATCH (n:Order)
    RETURN SUM(n.total_profit), n.sales_channel
    '''

quickCypher(driver, query3, verbose=True)

<Record SUM(n.total_profit)=16363385.660000004 n.sales_channel='Offline'>
<Record SUM(n.total_profit)=24963806.89 n.sales_channel='Online'>


In [6]:
# Close neo4j connection
driver.close()